<a href="https://colab.research.google.com/github/Mrcl3/O6EQTwpkNwvulbOa/blob/master/talent_search_with_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!pip install unidecode
!pip install gensim
!pip install transformers
!pip install sentence-transformers



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from unidecode import unidecode
import string
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read the data

In [70]:
df = pd.read_csv('/content/drive/MyDrive/Aprziva/O6EQTwpkNwvulbOa/potential-talents - Aspiring human resources - seeking human resources.csv')

## Define the sentence to be checked

# Preprocessing

In [71]:
df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [72]:
df.shape

(104, 5)

In [73]:
df.duplicated().sum()

0

In [74]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode

def preprocess(sentence):
    # Converting the words to lowercase
    sentence = sentence.lower()
    
    # Collecting a list of stop words from nltk and punctuation from the string class
    stopset = stopwords.words('english') + list(string.punctuation)
    
    # Remove stop words and punctuations from the string
    words = word_tokenize(sentence)
    words = [i for i in words if i.lower() not in stopset and not any(char in string.punctuation for char in i)]
    new_sentence = " ".join(words)
    
    # Remove non-ASCII characters
    new_sentence = unidecode(new_sentence)
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(new_sentence)
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return " ".join(words)



In [75]:
df['job_title'] = df['job_title'].apply(preprocess)


In [76]:
df['job_title'] 

0      2019 bauer college business graduate magna cum...
1      native english teacher epik english program korea
2                   aspiring human resource professional
3                    people development coordinator ryan
4           advisory board member celal bayar university
                             ...                        
99     aspiring human resource manager graduating may...
100                    human resource generalist loparex
101             business intelligence analytics traveler
102                                   always set success
103           director administration excellence logging
Name: job_title, Length: 104, dtype: object

# Word2Vec model

In [77]:
sentences = []
for item in df['job_title'].values:
  # Tokenize the sentence into a list of words
  words = nltk.word_tokenize(item)
    
  # Add the list of words to the sentences list
  sentences.append(words)
  model= Word2Vec(sentences,min_count=1,vector_size=300,workers=4)



In [78]:
print(model)

Word2Vec<vocab=172, vector_size=300, alpha=0.025>


In [79]:
model.save("word2vec.model")
model.save("model.bin")

In [80]:
model = Word2Vec.load('model.bin')

# Using the model to calculate similarities

In [81]:
similarity_array = []
# Define two example sentences
sentence2 = "aspiring human resources"
for item in df['job_title'].values:
  # Tokenize the sentences into lists of words
  tokens1 = nltk.word_tokenize(item)
  tokens2 = nltk.word_tokenize(sentence2)

  # Calculate the similarity between the two sentences
  similarity = model.wv.n_similarity(tokens1, tokens2)
  similarity_array.append(similarity)
df['similarities'] = similarity_array

In [82]:
df.tail(5)

,id,job_title,location,connection,fit,similarities
99,100,aspiring human resource manager graduating may...,"Cape Girardeau, Missouri",103,NaN,0.565562
100,101,human resource generalist loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,0.409442
101,102,business intelligence analytics traveler,Greater New York City Area,49,NaN,0.062707
102,103,always set success,Greater Los Angeles Area,500+,NaN,0.000582
103,104,director administration excellence logging,"Katy, Texas",500+,NaN,0.073247


# Basic fit ranking (similarities + connection scaled to (0,1)

In [83]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['similarities', 'connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])
print(df['connection'])
# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking'] = 0.8*df['similarities'] * 0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking'] = scaler.fit_transform(df['ranking'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking']])


0       85
1      500
2       44
3      500
4      500
      ... 
99     103
100    500
101     49
102    500
103    500
Name: connection, Length: 104, dtype: int64
                                             job_title   ranking
0    2019 bauer college business graduate magna cum...  0.165205
1    native english teacher epik english program korea  0.273624
2                 aspiring human resource professional  0.116683
3                  people development coordinator ryan  0.000000
4         advisory board member celal bayar university  0.136236
..                                                 ...       ...
99   aspiring human resource manager graduating may...  0.211271
100                  human resource generalist loparex  0.773300
101           business intelligence analytics traveler  0.018782
102                                 always set success  0.091685
103         director administration excellence logging  0.212826

[104 rows x 2 columns]


In [84]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.724779,0.165205
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.202076,0.273624
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,1.000000,0.116683
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.000000,0.000000
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.100613,0.136236


# Job titles ranking

In [85]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking', 'similarities', 'connection']])

                                             job_title   ranking  \
28   aspiring human resource management student see...  1.000000   
26   aspiring human resource management student see...  1.000000   
66     human resource staffing recruiting professional  0.797249   
100                  human resource generalist loparex  0.773300   
70           human resource generalist scottmadden inc  0.729565   
..                                                 ...       ...   
48                  aspiring human resource specialist  0.000000   
46                 people development coordinator ryan  0.000000   
59                  aspiring human resource specialist  0.000000   
35                  aspiring human resource specialist  0.000000   
21                 people development coordinator ryan  0.000000   

     similarities  connection  
28       0.738516         1.0  
26       0.738516         1.0  
66       0.588782         1.0  
100      0.571095         1.0  
70       0.538796      

# Try with BERT

In [86]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Selected words for similarity comparison
selected_words = ['aspiring', 'human', 'resource']

# Tokenize and encode all sentences and selected words together
tokenized_texts = tokenizer(df['job_title'].tolist(), padding=True, truncation=True, return_tensors='pt')
tokenized_selected_words = tokenizer(selected_words, padding=True, truncation=True, return_tensors='pt')

# Obtain BERT embeddings for all tokens
with torch.no_grad():
    outputs = model(**tokenized_texts)
    sentence_embeddings = outputs.last_hidden_state[:, 0, :].numpy()  # Sentence embeddings

    outputs_selected_words = model(**tokenized_selected_words)
    word_embeddings = outputs_selected_words.last_hidden_state[:, 0, :].numpy()  # Word embeddings

# Calculate cosine similarity between sentence embedding and word embeddings
similarity_scores = []
for i in range(len(df)):
    sentence_embedding = sentence_embeddings[i]
    scores = {}
    for j, word in enumerate(selected_words):
        word_embedding = word_embeddings[j]
        score = cosine_similarity(sentence_embedding.reshape(1, -1), word_embedding.reshape(1, -1)).item()
        scores[word] = score
    similarity_scores.append(scores)

# Reshape similarity scores and convert to DataFrame
similarity_scores = pd.DataFrame(similarity_scores, index=range(len(df)))
similarity_scores.columns = selected_words

# Merge similarity scores with the original DataFrame
df = pd.concat([df, similarity_scores], axis=1)

print(df)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


      id                                          job_title  \
0      1  2019 bauer college business graduate magna cum...   
1      2  native english teacher epik english program korea   
2      3               aspiring human resource professional   
3      4                people development coordinator ryan   
4      5       advisory board member celal bayar university   
..   ...                                                ...   
99   100  aspiring human resource manager graduating may...   
100  101                  human resource generalist loparex   
101  102           business intelligence analytics traveler   
102  103                                 always set success   
103  104         director administration excellence logging   

                                location  connection  fit  similarities  \
0                         Houston, Texas    0.168337  NaN      0.724779   
1                                 Kanada    1.000000  NaN      0.202076   
2    Raleigh-Durha

In [87]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.724779,0.165205,0.802823,0.849149,0.811189
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.202076,0.273624,0.852012,0.874882,0.874288
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,1.000000,0.116683,0.760295,0.807430,0.757689
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.000000,0.000000,0.853755,0.899812,0.868571
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.100613,0.136236,0.794921,0.843250,0.816467


In [88]:
df['avarage'] = (df['aspiring'] + df['human'] + df['resource'])/3

In [89]:
df.head()

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource,avarage
0,1,2019 bauer college business graduate magna cum...,"Houston, Texas",0.168337,NaN,0.724779,0.165205,0.802823,0.849149,0.811189,0.821054
1,2,native english teacher epik english program korea,Kanada,1.000000,NaN,0.202076,0.273624,0.852012,0.874882,0.874288,0.867061
2,3,aspiring human resource professional,"Raleigh-Durham, North Carolina Area",0.086172,NaN,1.000000,0.116683,0.760295,0.807430,0.757689,0.775138
3,4,people development coordinator ryan,"Denton, Texas",1.000000,NaN,0.000000,0.000000,0.853755,0.899812,0.868571,0.874046
4,5,advisory board member celal bayar university,"İzmir, Türkiye",1.000000,NaN,0.100613,0.136236,0.794921,0.843250,0.816467,0.818213


In [90]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['avarage', 'connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])

# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking_BERT'] = 0.8*df['avarage'] * 0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking_BERT'] = scaler.fit_transform(df['ranking_BERT'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking','ranking_BERT']])


                                             job_title   ranking  ranking_BERT
0    2019 bauer college business graduate magna cum...  0.165205      0.094319
1    native english teacher epik english program korea  0.273624      0.745856
2                 aspiring human resource professional  0.116683      0.032324
3                  people development coordinator ryan  0.000000      0.774029
4         advisory board member celal bayar university  0.136236      0.548840
..                                                 ...       ...           ...
99   aspiring human resource manager graduating may...  0.211271      0.133589
100                  human resource generalist loparex  0.773300      0.839160
101           business intelligence analytics traveler  0.018782      0.053072
102                                 always set success  0.091685      0.855278
103         director administration excellence logging  0.212826      0.865002

[104 rows x 3 columns]


In [91]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking','ranking_BERT']])

                                             job_title   ranking  ranking_BERT
28   aspiring human resource management student see...  1.000000      0.399975
26   aspiring human resource management student see...  1.000000      0.399975
66     human resource staffing recruiting professional  0.797249      0.743885
100                  human resource generalist loparex  0.773300      0.839160
70           human resource generalist scottmadden inc  0.729565      0.822557
..                                                 ...       ...           ...
48                  aspiring human resource specialist  0.000000      0.000000
46                 people development coordinator ryan  0.000000      0.774029
59                  aspiring human resource specialist  0.000000      0.000000
35                  aspiring human resource specialist  0.000000      0.000000
21                 people development coordinator ryan  0.000000      0.774029

[104 rows x 3 columns]


In [92]:
# Sort the DataFrame in ascending order based on a column
df_sorted = df.sort_values('ranking_BERT', ascending=False)

# Print the sorted DataFrame
print(df_sorted[['job_title', 'ranking','ranking_BERT', 'connection']])

                                             job_title   ranking  \
67                 human resource specialist luxottica  0.712848   
77                    human resource generalist schwan  0.650043   
103         director administration excellence logging  0.212826   
102                                 always set success  0.091685   
64   human resource coordinator intercontinental bu...  0.651028   
..                                                 ...       ...   
48                  aspiring human resource specialist  0.000000   
5                   aspiring human resource specialist  0.000000   
23                  aspiring human resource specialist  0.000000   
59                  aspiring human resource specialist  0.000000   
35                  aspiring human resource specialist  0.000000   

     ranking_BERT  connection  
67       1.000000         1.0  
77       0.877701         1.0  
103      0.865002         1.0  
102      0.855278         1.0  
64       0.852687      

# SBERT

In [93]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')


In [94]:
sentences = list(df['job_title'].values)

In [95]:
# Encode the target sentence and other sentences
target_sentence = 'aspiring human resource'
target_embedding = model.encode([target_sentence])[0]
sentence_embeddings = model.encode(sentences)

# Calculate cosine similarity between the target sentence and other sentences
similarities = cosine_similarity([target_embedding], sentence_embeddings)[0]

# Sort the sentences based on their similarity scores
results = sorted(zip(sentences, similarities), key=lambda x: x[1], reverse=True)

# Print the most similar sentences
for sentence, similarity in results:
    print(f"Similarity: {similarity:.4f}\tSentence: {sentence}")

Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7982	Sentence: aspiring human resource professional
Similarity: 0.7820	Sentence: seeking human resource opportunity
Similarity: 0.7820	Sentence: seeking human resource opportunity
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7614	Sentence: aspiring human resource specialist
Similarity: 0.7600	Sentence: seeking human resource position
Similarity: 0.5887	Sentence: 

In [96]:
from sklearn.preprocessing import MinMaxScaler

#Select the input features for ranking
features = ['connection']

# Replace '500+' with 500 in the 'connections' column
df['connection'] = df['connection'].replace('500+ ', '500')

df['connection'] = pd.to_numeric(df['connection'])
print(df['connection'])
# Scale the 'connections' column to a range of 0 to 1
scaler = MinMaxScaler()
df['connection'] = scaler.fit_transform(df['connection'].values.reshape(-1, 1))

# Normalize the remaining features to a range of 0 to 1
df[features] = scaler.fit_transform(df[features])

# Calculate the ranking score
df['ranking_SBERT'] = 0.8*similarities*0.2*df['connection']

# Normalize the ranking score to a range of 0 to 1
df['ranking_SBERT'] = scaler.fit_transform(df['ranking_SBERT'].values.reshape(-1, 1))

# Print the ranked candidates
print(df[['job_title', 'ranking','ranking_SBERT']])


0      0.168337
1      1.000000
2      0.086172
3      1.000000
4      1.000000
         ...   
99     0.204409
100    1.000000
101    0.096192
102    1.000000
103    1.000000
Name: connection, Length: 104, dtype: float64
                                             job_title   ranking  \
0    2019 bauer college business graduate magna cum...  0.165205   
1    native english teacher epik english program korea  0.273624   
2                 aspiring human resource professional  0.116683   
3                  people development coordinator ryan  0.000000   
4         advisory board member celal bayar university  0.136236   
..                                                 ...       ...   
99   aspiring human resource manager graduating may...  0.211271   
100                  human resource generalist loparex  0.773300   
101           business intelligence analytics traveler  0.018782   
102                                 always set success  0.091685   
103         director administr

In [99]:
df.tail(40)

,id,job_title,location,connection,fit,similarities,ranking,aspiring,human,resource,avarage,ranking_BERT,ranking_SBERT
64,65,human resource coordinator intercontinental bu...,"Atlanta, Georgia",1.000000,NaN,0.480795,0.651028,0.908056,0.853949,0.918639,0.800821,0.852687,0.463957
65,66,experienced retail manager aspiring human reso...,"Austin, Texas Area",0.112224,NaN,0.738913,0.112285,0.656040,0.737290,0.653072,0.000000,0.000000,0.122969
66,67,human resource staffing recruiting professional,"Jackson, Mississippi Area",1.000000,NaN,0.588782,0.797249,0.847956,0.879553,0.872207,0.698637,0.743885,0.654748
67,68,human resource specialist luxottica,Greater New York City Area,1.000000,NaN,0.526450,0.712848,0.953045,0.873255,0.963917,0.939173,1.000000,0.450554
68,69,director human resource north america groupe b...,"Greater Grand Rapids, Michigan Area",1.000000,NaN,0.534259,0.723422,0.824047,0.863262,0.844315,0.612661,0.652341,0.624740
69,70,retired army national guard recruiter office m...,"Virginia Beach, Virginia",0.162325,NaN,0.392643,0.086302,0.766718,0.849971,0.790793,0.455915,0.078799,0.152845
70,71,human resource generalist scottmadden inc,"Raleigh-Durham, North Carolina Area",1.000000,NaN,0.538796,0.729565,0.865610,0.903819,0.888803,0.772524,0.822557,0.464661
71,72,business management major aspiring human resou...,"Monroe, Louisiana Area",0.008016,NaN,0.800839,0.008692,0.757936,0.780612,0.756592,0.314067,0.002681,0.069493
72,73,aspiring human resource manager seeking intern...,"Houston, Texas Area",0.012024,NaN,0.877091,0.014280,0.805727,0.841271,0.814900,0.524623,0.006717,0.075727
73,74,human resource professional,Greater Boston Area,0.030060,NaN,0.684059,0.027844,0.877678,0.904746,0.900114,0.803212,0.025708,0.090907
